Hola !

Mi nombre es Oscar Flores y me toca revisar tu proyecto de hoy. Si tienes algún comentario que quieras agregar en tus respuestas te puedes referir a mi como Oscar, no hay problema que me trates de tú.

Si veo un error en la primera revisión solamente lo señalaré y dejaré que tú encuentres de qué se trata y cómo arreglarlo. Debo prepararte para que te desempeñes como especialista en Data, en un trabajo real, el responsable a cargo tuyo hará lo mismo. Si aún tienes dificultades para resolver esta tarea, te daré indicaciones más precisas en una siguiente iteración.

Te dejaré mis comentarios más abajo - **por favor, no los muevas, modifiques o borres**

Comenzaré mis comentarios con un resumen de los puntos que están bien, aquellos que debes corregir y aquellos que puedes mejorar. Luego deberás revisar todo el notebook para leer mis comentarios, los cuales estarán en rectángulos de color verde, amarillo o rojo como siguen:

<div class="alert alert-block alert-success">
<b>Comentario de Reviewer</b> <a class="tocSkip"></a>
    
Muy bien! Toda la respuesta fue lograda satisfactoriamente.
</div>

<div class="alert alert-block alert-warning">
<b>Comentario de Reviewer</b> <a class="tocSkip"></a>

Existen detalles a mejorar. Existen recomendaciones.
</div>

<div class="alert alert-block alert-danger">

<b>Comentario de Reviewer</b> <a class="tocSkip"></a>

Se necesitan correcciones en el bloque. El trabajo no puede ser aceptado con comentarios en rojo sin solucionar.
</div>

Cualquier comentario que quieras agregar entre iteraciones de revisión lo puedes hacer de la siguiente manera:

<div class="alert alert-block alert-info">
<b>Respuesta estudiante.</b> <a class="tocSkip"></a>
</div>


## Resumen de la revisión 1 <a class="tocSkip"></a>

<div class="alert alert-block alert-success">
<b>Comentario de Reviewer</b> <a class="tocSkip"></a>

Excelente trabajo! La verdad no tengo nada que señalar para que debas corregir, todo está muy bien. Tal vez le agregaría más análisis al inicio, pero solo eso. Felicitaciones!
</div>

---

#### Introducción

Los dueños de OilyGiant están interesados en encontrar el mejor lugar para un nuevo pozo utilizando los parámetros recolectados de los pozos de petróleo en tres regiones diferentes. Se requiere construir un modelo para predecir el volumen de reservas en los nuevos pozos y así, seleccionar los pozos de petróleo con los beneficios estimados más altos.

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy import stats as st
from joblib import dump

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_curve
from sklearn.preprocessing import OrdinalEncoder
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

#### Preparación de datos

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv') 
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
data_0.sort_values(by='product', ascending = False).head()

,id,f0,f1,f2,product
8826,rjMou,1.797736,0.098212,6.148260,185.364347
99818,7cHIv,0.518445,-0.412790,4.951916,185.362690
94175,uCDzR,0.351428,-0.400244,7.227618,185.355615
1925,IfqrC,0.624430,-0.469312,5.753677,185.354980
45291,5FEPb,1.758787,-0.395038,6.719085,185.352015


In [5]:
data_1.sort_values(by='product', ascending = False).head()

,id,f0,f1,f2,product
53864,MzRzn,2.901352,-3.475398,5.001393,137.945408
97083,nW6eC,-4.030430,-14.020643,5.009571,137.945408
88340,xwJzQ,-3.373117,-9.227661,4.994369,137.945408
64879,Xd8DC,-5.360281,-2.388204,5.001439,137.945408
7288,Stdrb,-8.058661,-2.244840,5.000753,137.945408


In [6]:
data_2.sort_values(by='product', ascending = False).head()

,id,f0,f1,f2,product
79705,UAhji,-2.747914,1.555227,3.342182,190.029838
93444,IB0JE,3.026506,1.344623,8.891243,190.013589
35099,2HeCn,2.848844,-1.701996,9.437101,190.011722
21943,dldNH,-2.500091,2.024392,6.291513,190.010982
37870,CpuBi,-0.686667,-3.219452,5.963290,190.010029


Los datos están listos para ser utilizados en el entranamiento de los modelos para cada región. No es necesario hacerse cargo de datos faltantes o datos sin formato adecuado.

<div class="alert alert-block alert-success">
<b>Comentario de Reviewer</b> <a class="tocSkip"></a>

Correcto, los datos están completos
</div>

<div class="alert alert-block alert-warning">
<b>Comentario de Reviewer</b> <a class="tocSkip"></a>

Ojo con verificar si hay valores duplicados. Además, graficar histogramas o scatter plot de las variables puede servir para generar más entendimiento de los datos.
</div>

#### Entrenamiento del modelo para cada región

El modelo más conveniente para entrenar es el de regresión lineal debido al tipo de datos recopilados en features y target.

In [7]:
target_0 = data_0['product']
features_0 = data_0.drop(['id','product'], axis=1)

In [8]:
target_1 = data_1['product']
features_1 = data_1.drop(['id','product'], axis=1)

In [9]:
target_2 = data_2['product']
features_2 = data_2.drop(['id','product'], axis=1)

Los datos colectados de características de los pozos y volumen estimado serán utilizados para generar el conjunto de entrenamiento y el conjunto de validación en una proporción de 75:25.

In [10]:
data_0_train_features, data_0_valid_features, data_0_train_target, data_0_valid_target = train_test_split(features_0, target_0, test_size=0.25, random_state=12345) 
data_1_train_features, data_1_valid_features, data_1_train_target, data_1_valid_target = train_test_split(features_1, target_1, test_size=0.25, random_state=12345) 
data_2_train_features, data_2_valid_features, data_2_train_target, data_2_valid_target = train_test_split(features_2, target_2, test_size=0.25, random_state=12345) 

In [11]:
model = LinearRegression()

model.fit(data_0_train_features, data_0_train_target)
model.fit(data_1_train_features, data_1_train_target)
model.fit(data_2_train_features, data_2_train_target)

predictions_valid_0 = model.predict(data_0_valid_features)
predictions_valid_1 = model.predict(data_1_valid_features)
predictions_valid_2 = model.predict(data_2_valid_features)

<div class="alert alert-block alert-success">
<b>Comentario de Reviewer</b> <a class="tocSkip"></a>

Muy bien, correcta la divisón de datos y predicciones
</div>

El conjunto de datos de validación proporciona la base para los datos de predicción que se utilizarán en los pasos siguientes para cálculos de ganancias potenciales por región.

##### Cálculo del volumen promedio y RECM del modelo

Se calcula la media de los datos de validación para identificar la diferencia promedio en el volumen de reservas de acuerdo con el modelo para cada región.

In [12]:
target_mean_0 = predictions_valid_0.mean()
target_mean_1 = predictions_valid_1.mean()
target_mean_2 = predictions_valid_2.mean()
print("Volumen promedio previsto de las reservas de la región 0:", target_mean_0)
print("Volumen promedio previsto de las reservas de la región 1:", target_mean_1)
print("Volumen promedio previsto de las reservas de la región 2:", target_mean_2)

Volumen promedio previsto de las reservas de la región 0: 95.01224887602382
Volumen promedio previsto de las reservas de la región 1: 95.2329505925182
Volumen promedio previsto de las reservas de la región 2: 94.96504596800489


Se calcular la RECM en los datos de validación para identificar la dispersión promedio del volumen de reservas de acuerdo con el modelo para cada región.

In [13]:
RECM_0 = mean_squared_error(data_0_valid_target, predictions_valid_0)**0.5
RECM_1 = mean_squared_error(data_1_valid_target, predictions_valid_1)**0.5
RECM_2 = mean_squared_error(data_2_valid_target, predictions_valid_2)**0.5
print("RECM del modelo de regresión lineal de la región 0:", RECM_0)
print("RECM del modelo de regresión lineal de la región 1:", RECM_1)
print("RECM del modelo de regresión lineal de la región 2:", RECM_2)

RECM del modelo de regresión lineal de la región 0: 38.94065197644272
RECM del modelo de regresión lineal de la región 1: 44.94590426427794
RECM del modelo de regresión lineal de la región 2: 40.02970873393434


Las tres regiones tienen un promedio de volumen muy similar, siendo la región 2 la de menor volumen. Sin embargo, el RECM difiere entre las regiones siendo el más alto el de la región 1, indicando que el valor predicho del modelo se aleja del valor real de los datos recopilados, o bien, que existe mayor dispersión de los datos del modelo alrededor de la línea del modelo de regresión lineal.

<div class="alert alert-block alert-success">
<b>Comentario de Reviewer</b> <a class="tocSkip"></a>

Correcto
</div>

#### Cálculo de ganancias con el volumen promedio de 200 pozos por región

In [14]:
drilling_budget_000 = 100000
max_capacity = 200
sales_price = 4.5

##### Ventas promedio por región

In [15]:
total_average_sales_0 = target_mean_0 * sales_price * max_capacity
total_average_sales_1 = target_mean_1 * sales_price * max_capacity
total_average_sales_2 = target_mean_2 * sales_price * max_capacity

In [16]:
print("Ventas con el volumen promedio previsto de la región 0:", total_average_sales_0)
print("Ventas con el volumen promedio previsto de la región 1:", total_average_sales_1)
print("Ventas con el volumen promedio previsto de la región 2:", total_average_sales_2)

Ventas con el volumen promedio previsto de la región 0: 85511.02398842144
Ventas con el volumen promedio previsto de la región 1: 85709.65553326637
Ventas con el volumen promedio previsto de la región 2: 85468.5413712044


<div class="alert alert-block alert-success">
<b>Comentario de Reviewer</b> <a class="tocSkip"></a>

Ok, correcto para el promedio.
</div>

##### Ganancias promedio por región

In [17]:
gross_profit_0 = total_average_sales_0 - drilling_budget_000
gross_profit_1 = total_average_sales_1 - drilling_budget_000
gross_profit_2 = total_average_sales_2 - drilling_budget_000

In [18]:
print("Ganancias con el volumen promedio de la región 0:", gross_profit_0)
print("Ganancias con el volumen promedio de la región 1:", gross_profit_1)
print("Ganancias con el volumen promedio de la región 2:", gross_profit_2)

Ganancias con el volumen promedio de la región 0: -14488.976011578561
Ganancias con el volumen promedio de la región 1: -14290.34446673363
Ganancias con el volumen promedio de la región 2: -14531.458628795604


Ninguna región es rentable considerando el volumen promedio de 200 pozos en cada región.

<div class="alert alert-block alert-success">
<b>Comentario de Reviewer</b> <a class="tocSkip"></a>

Claro, considerando que el promedio incluye peores y mejores.
</div>

##### Volumen de reservas suficiente para desarrollar pozos sin pérdidas

A continuación se realiza el cálculo del volumen promedio de cada pozo para que al tomar 200 de ellos no se genere pérdida.

In [19]:
min_total_capacity = drilling_budget_000 / sales_price

In [20]:
print('Volumen total mínimo de reservas para evitar pérdida', min_total_capacity)

Volumen total mínimo de reservas para evitar pérdida 22222.222222222223


In [21]:
min_target_mean = drilling_budget_000 / (sales_price * max_capacity)

In [22]:
print("Volumen promedio mínimo de las reservas para evitar pérdidas:", min_target_mean)

Volumen promedio mínimo de las reservas para evitar pérdidas: 111.11111111111111


Para que las regiones fueran rentables, se requeriría que el volumen promedio mínimo de cada pozo fuera de 111,111.

<div class="alert alert-block alert-success">
<b>Comentario de Reviewer</b> <a class="tocSkip"></a>

Correctos los resultados
</div>

#### Cálculo de ganancia con selección de los 200 pozos más abundantes

A continuación se calcula la cantidad de ganancia y el margen que se genera cuando se escogen deliberadamente los mejores 200 pozos en cada región, utilizando los datos de predicción del modelo.

In [23]:
predictions_region_0 = pd.DataFrame(data = predictions_valid_0, columns = ['predicted_product_region_0']).sort_values(by= 'predicted_product_region_0', ascending=False)
predictions_region_1 = pd.DataFrame(data = predictions_valid_1, columns = ['predicted_product_region_1']).sort_values(by= 'predicted_product_region_1', ascending=False)
predictions_region_2 = pd.DataFrame(data = predictions_valid_2, columns = ['predicted_product_region_2']).sort_values(by= 'predicted_product_region_2', ascending=False)

In [24]:
predictions_region_0 = predictions_region_0.reset_index(drop=True)
predictions_region_1 = predictions_region_1.reset_index(drop=True)
predictions_region_2 = predictions_region_2.reset_index(drop=True)

In [25]:
total_max_sales_0 = sales_price * predictions_region_0.loc[:200,'predicted_product_region_0'].sum()
total_max_sales_1 = sales_price * predictions_region_1.loc[:200,'predicted_product_region_1'].sum()
total_max_sales_2 = sales_price * predictions_region_2.loc[:200,'predicted_product_region_2'].sum()

In [26]:
projected_gross_profit_0 = total_max_sales_0 - drilling_budget_000
projected_gross_profit_1 = total_max_sales_1 - drilling_budget_000
projected_gross_profit_2 = total_max_sales_2 - drilling_budget_000

In [27]:
print("Ganancias estimadas con evaluación de la región 0:", projected_gross_profit_0)
print("Ganancias estimadas con evaluación de la región 1:", projected_gross_profit_1)
print("Ganancias estimadas con evaluación de la región 2:", projected_gross_profit_2)

Ganancias estimadas con evaluación de la región 0: 31148.294175369374
Ganancias estimadas con evaluación de la región 1: -326.74118252946937
Ganancias estimadas con evaluación de la región 2: 33858.35894855796


In [28]:
margin_to_sales_0 = projected_gross_profit_0 / total_max_sales_0
margin_to_sales_1 = projected_gross_profit_1 / total_max_sales_1
margin_to_sales_2 = projected_gross_profit_2 / total_max_sales_2

In [29]:
print('Margen de ganancia/pérdida estimado de la región 0:', margin_to_sales_0*100)
print('Margen de ganancia/pérdida estimado de la región 1:', margin_to_sales_1*100)
print('Margen de ganancia/pérdida estimado de la región 2:', margin_to_sales_2*100)

Margen de ganancia/pérdida estimado de la región 0: 23.750437907883406
Margen de ganancia/pérdida estimado de la región 1: -0.32781228025043646
Margen de ganancia/pérdida estimado de la región 2: 25.29416856333178


<div class="alert alert-block alert-success">
<b>Comentario de Reviewer</b> <a class="tocSkip"></a>

Muy bien
</div>

Podemos observar que la región con las ganancias potenciales más altas es la región 2, y la región con pérdidas potenciales más grandes es la región 1.

#### Cálculo de riesgos y ganancias utilizando bootstrapping

A continuación se calculará el promedio de 1000 corridas de las ganancias de los mejores 200 pozos una vez que el modelo ha evaluado 500 puntos aleatorios.

In [30]:
def profit(volume, budget, top_wells):
    volume_sorted = volume.sort_values(ascending=False)
    selected_volume = volume_sorted[:top_wells]
    return sales_price * selected_volume.sum() - budget

In [31]:
state = np.random.RandomState(12345)
profit_0 = []
profit_1 = []
profit_2 = []

for i in range(1000):
    
    subsample_0 = predictions_region_0['predicted_product_region_0'].sample(n=500, replace=True, random_state=state)
    subsample_1 = predictions_region_1['predicted_product_region_1'].sample(n=500, replace=True, random_state=state)
    subsample_2 = predictions_region_2['predicted_product_region_2'].sample(n=500, replace=True, random_state=state)
    
    profit_0.append(profit(subsample_0, drilling_budget_000, 200))
    profit_1.append(profit(subsample_1, drilling_budget_000, 200))
    profit_2.append(profit(subsample_2, drilling_budget_000, 200))
    
profit_0 = pd.Series(profit_0)
profit_1 = pd.Series(profit_1)
profit_2 = pd.Series(profit_2)

In [32]:
lower_0 = profit_0.quantile(0.05)
mean_0 = profit_0.mean()
upper_0 = profit_0.quantile(0.95)

lower_1 = profit_1.quantile(0.05)
mean_1 = profit_1.mean()
upper_1 = profit_1.quantile(0.95)

lower_2 = profit_2.quantile(0.05)
mean_2 = profit_2.mean()
upper_2 = profit_2.quantile(0.95)

In [33]:
print('Límite inferior del intervalo de confianza del 95% :', lower_0)
print('Beneficio total medio de los 200 mejores pozos     :', mean_0)
print('Límite superior del intervalo de confianza del 95% :', upper_0)

Límite inferior del intervalo de confianza del 95% : -25.54942155009412
Beneficio total medio de los 200 mejores pozos     : 1556.459524856751
Límite superior del intervalo de confianza del 95% : 3109.8850240197585


Para la región 0 existe la posibilidad de obtener pérdidas al excavar en los mejores 200 puntos de 500 evaluados por lo que no es conveniente trabajar en esta zona.

In [34]:
print('Límite inferior del intervalo de confianza del 95% :', lower_1)
print('Beneficio total medio de los 200 mejores pozos     :', mean_1)
print('Límite superior del intervalo de confianza del 95% :', upper_1)

Límite inferior del intervalo de confianza del 95% : -5995.832976850825
Beneficio total medio de los 200 mejores pozos     : -5273.866823644733
Límite superior del intervalo de confianza del 95% : -4530.039093315605


Para la región 1 existe el 100% de posibilidad de obtener pérdidas al excavar en los mejores 200 puntos de 500 evaluados por lo que no es recomendable trabajar en esta zona.

In [35]:
print('Límite inferior del intervalo de confianza del 95% :', lower_2)
print('Beneficio total medio de los 200 mejores pozos     :', mean_2)
print('Límite superior del intervalo de confianza del 95% :', upper_2)

Límite inferior del intervalo de confianza del 95% : 1077.3237722931335
Beneficio total medio de los 200 mejores pozos     : 2693.4637347394973
Límite superior del intervalo de confianza del 95% : 4322.449298801707


Para la región 2 existe 0% de posibilidad de obtener pérdidas al excavar en los mejores 200 puntos de 500 evaluados por lo que es recomendable trabajar en esta zona.

<div class="alert alert-block alert-success">
<b>Comentario de Reviewer</b> <a class="tocSkip"></a>

Correcto con el procedimiento de bootstrapping
</div>

##### Riesgo de pérdidas utilizando la media y desviación estándar muestrales

A continuación se calculará el intervalo de confianza de cada una de los promedios de ganancia en cada región, para determinar el riesgo de pérdida en cada una de ellas.

In [36]:
confidence_interval_0 = st.t.interval(
    0.95, len(profit_0)-1, loc=profit_0.mean(), scale=profit_0.sem())

print('Intervalo de confianza del 95 %:', confidence_interval_0)

Intervalo de confianza del 95 %: (1497.3336782360707, 1615.5853714774312)


El 5% de las veces que se corra esta prueba, la media poblacional no estará contenida dentro del intervalo de confianza de ganancias entre $1497 y $1615. Lo que quiere decir que las ganancias serán más altas o bajas que los límites de confianza.

In [37]:
confidence_interval_1 = st.t.interval(
    0.95, len(profit_1)-1, loc=profit_1.mean(), scale=profit_1.sem())

print('Intervalo de confianza del 95 %:', confidence_interval_1)

Intervalo de confianza del 95 %: (-5300.887959595282, -5246.845687694184)


En esta región particular, no es conveniente realizar ningún pozo. De acuerdo con el modelo, las exploraciones proporcionarán pozos de muy bajo volumen que no justifican el costo de excavación de los 200 mejores pozos usando 500 pozos aleatorios.

In [38]:
confidence_interval_2 = st.t.interval(
    0.95, len(profit_2)-1, loc=profit_2.mean(), scale=profit_2.sem())

print('Intervalo de confianza del 95 %:', confidence_interval_2)

Intervalo de confianza del 95 %: (2632.5960136380104, 2754.3314558409843)


El 5% de las veces que se corra esta prueba, la media poblacional no estará contenida dentro del intervalo de confianza de ganancias entre $2632 y $2754. Lo que quiere decir que las ganancias serán más altas o bajas que los límites de confianza.

La región de exploración que mejores ganancias proporcionará de acuerdo con el modelo es la región 2, cada vez que se realicen excavaciones en puntos aleatorios es altamente probable que los 200 mejores pozos proporcionen ganancias por encima del costo de excavación.

<div class="alert alert-block alert-success">
<b>Comentario de Reviewer</b> <a class="tocSkip"></a>

Excelente!
</div>